In [1]:
import json
from pathlib import Path
from pprint import pprint, pformat

In [2]:
dataset_path = Path('../data/Frames-dataset/frames.json')

In [3]:
with open(dataset_path, 'r') as data_file:    
    data = json.load(data_file)

In [4]:
data[0]['turns'][2].keys(), data[0]['turns'][3].keys(), 

(dict_keys(['text', 'labels', 'author', 'timestamp']),
 dict_keys(['db', 'text', 'labels', 'author', 'timestamp']))

In [5]:
RED = '\x1b[31m'
BLUE = '\x1b[34m'
NC = '\x1b[0m'

def show_turn(dial_id, turn_id, filt='11111'):
    return '\n'.join(filter(None, [
        '' if filt[0] == '0' else '{}User   (raw):\n{}\n{}'.
            format(RED, data[dial_id]['turns'][turn_id*2]['text'], NC),
        '' if filt[2] == '0' else '{}System (raw):\n{}\n{}'.
            format(BLUE, data[dial_id]['turns'][turn_id*2+1]['text'], NC),
    ]))

In [6]:
def text_iterator(dials):
    for dial_id, dial in enumerate(dials):
        for turn_id, turn in enumerate(dial['turns']):
            yield turn['text']

def act_iterator(dials, authors=['user']):
    for dial_id, dial in enumerate(dials):
        for turn_id, turn in enumerate(dial['turns']):
            if turn['author'] in authors:
                for act in turn['labels']['acts']:
                    yield act

In [8]:
data[0].keys()

dict_keys(['user_id', 'turns', 'wizard_id', 'id', 'labels'])

In [12]:
users = {}
wizards = {}
for dial_id, dial in enumerate(data):
    cnt = users.get(dial['user_id'], 0) + 1
    users[dial['user_id']] = cnt
    
    cnt = wizards.get(dial['wizard_id'], 0) + 1
    wizards[dial['wizard_id']] = cnt

In [14]:
users

{'U21E41CQP': 13,
 'U21RP4FCY': 77,
 'U22HTHYNP': 69,
 'U22K1SX9N': 345,
 'U231PNNA3': 122,
 'U23KPC9QV': 27,
 'U23KR88NT': 35,
 'U24V2QUKC': 323,
 'U260BGVS6': 81,
 'U2709166N': 214,
 'U2AMZ8TLK': 63}

In [15]:
wizards

{'U1Z7FQTHC': 32,
 'U21DKG18C': 14,
 'U21DMV0KA': 288,
 'U21E0179B': 283,
 'U21E41CQP': 64,
 'U21T9NMKM': 301,
 'U22HTHYNP': 37,
 'U23KPC9QV': 60,
 'U23KR88NT': 38,
 'U23KS7P33': 49,
 'U260BGVS6': 144,
 'U28KSAJBY': 59}

In [70]:
for s, _ in zip(text_iterator(data), range(20)):
    print(s)

I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.
Hi...I checked a few options for you, and unfortunately, we do not currently have any trips that meet this criteria.  Would you like to book an alternate travel option?
Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.
I checked the availability for this date and there were no trips available.  Would you like to select some alternate dates?
I have no flexibility for dates... but I can leave from Atlantis rather than Caprica. How about that?
I checked the availability for that date and there were no trips available.  Would you like to select some alternate dates?
I suppose I'll speak with my husband to see if we can choose other dates, and then I'll come back to you.Thanks for your help
Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.
Hi. Sorry, I can't find any tri

In [46]:
slot_types = [
    'count',
    'count_amenities',
    'count_name',
    'count_dst_city',
    'count_seat',
    'count_category',
    'id',
    'vicinity',
    'amenities',
    'ref_anaphora',
    'imple_anaphora',
    'ref',
    'read',
    'write',
    'intent',
    'action',
]

In [73]:
for turn_id, turn in enumerate(data[1]['turns']):
    print('turn_id = {:2d}, active frame = {:2d}\n{}\n'.format(
        turn_id, turn['labels']['active_frame'], turn['text']))

turn_id =  0, active frame =  1
Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.

turn_id =  1, active frame =  1
Hi. Sorry, I can't find any trips from Gotham City to Mos Eisley for you.

turn_id =  2, active frame =  2
What about a trip from Gotham City to Neverland for the same budget?

turn_id =  3, active frame =  2
Sorry, I cannot find any trips leaving from Gotham City. Could you suggest another nearby departure city?

turn_id =  4, active frame =  3
Would any packages to Mos Eisley be available if I increase my budget to $2500?

turn_id =  5, active frame =  3
There are no trips available to Mos Eisley.

turn_id =  6, active frame =  4
You know what, I'd like to try and visit Neverland

turn_id =  7, active frame =  4
I cannot find any trips available to Neverland.

turn_id =  8, active frame =  5
Do you have any trips from Gotham City to Kobe for my original budget of $2100?

turn_id =  9, active frame =  5
I can book you a 3 day trip to Kobe le

### Actions

In [53]:
act_dict = {}
# for act in act_iterator(data, authors=['user', 'wizard']):
for act in act_iterator(data):
    args = act_dict.get(act['name'], set())
    for arg in act['args']:
#         assert arg['key'] in slot_types, (arg['key'], act)
        args.add(arg['key'])
    act_dict[act['name']] = args

In [56]:
for name, slots in act_dict.items():
    if 'id' in slots:
        print(name)

In [54]:
pprint(act_dict)

{'affirm': {'ref'},
 'confirm': {'action',
             'beach',
             'breakfast',
             'budget',
             'budget_ok',
             'category',
             'count',
             'count_dst_city',
             'count_name',
             'count_seat',
             'dst_city',
             'duration',
             'end_date',
             'gst_rating',
             'intent_ok',
             'n_adults',
             'n_adults_ok',
             'n_children',
             'name',
             'or_city',
             'park',
             'price',
             'ref',
             'seat',
             'seat_ok',
             'str_date',
             'wifi'},
 'goodbye': set(),
 'greeting': set(),
 'inform': {'amenities',
            'beach',
            'breakfast',
            'budget',
            'budget_ok',
            'category',
            'count_dst_city',
            'count_name',
            'downtown',
            'dst_city',
            'end_date',
           

In [43]:
# acts, name and args
for act, _ in zip(act_iterator(data), range(10)):
    print('name: {}\nargs: {}\n'.format(
        act['name'], act['args']))

name: no_result
args: [{'val': [{'annotations': [], 'frame': 1, 'fromrange': False}], 'key': 'ref'}]

name: suggest
args: [{'key': 'dst_city'}]

name: sorry
args: []

name: no_result
args: [{'val': [{'annotations': [{'val': None, 'key': 'str_date'}], 'frame': 2, 'fromrange': False}], 'key': 'ref'}]

name: suggest
args: [{'key': 'str_date'}, {'key': 'end_date'}]

name: no_result
args: [{'val': [{'annotations': [{'val': None, 'key': 'str_date'}], 'frame': 3, 'fromrange': False}], 'key': 'ref'}]

name: suggest
args: [{'key': 'str_date'}, {'key': 'end_date'}]

name: sorry
args: []

name: no_result
args: [{'val': [{'annotations': [{'val': 'Mos Eisley', 'key': 'dst_city'}], 'frame': 1, 'fromrange': False}, {'annotations': [{'val': 'Gotham City', 'key': 'or_city'}], 'frame': 1, 'fromrange': False}], 'key': 'ref'}]

name: sorry
args: []



In [23]:
# acts, raw
cnt = 0
for dial_id, dial in enumerate(data):
    cnt += 1
    if cnt >= 3:
        break
    
    for turn_id, turn in enumerate(dial['turns']):
        if turn_id % 2:  # wizard
            for act in turn['labels']['acts']:
                print(act)
        else:  # user
            pass

{'args': [{'val': [{'annotations': [], 'frame': 1, 'fromrange': False}], 'key': 'ref'}], 'name': 'no_result'}
{'args': [{'key': 'dst_city'}], 'name': 'suggest'}
{'args': [], 'name': 'sorry'}
{'args': [{'val': [{'annotations': [{'val': None, 'key': 'str_date'}], 'frame': 2, 'fromrange': False}], 'key': 'ref'}], 'name': 'no_result'}
{'args': [{'key': 'str_date'}, {'key': 'end_date'}], 'name': 'suggest'}
{'args': [{'val': [{'annotations': [{'val': None, 'key': 'str_date'}], 'frame': 3, 'fromrange': False}], 'key': 'ref'}], 'name': 'no_result'}
{'args': [{'key': 'str_date'}, {'key': 'end_date'}], 'name': 'suggest'}
{'args': [], 'name': 'sorry'}
{'args': [{'val': [{'annotations': [{'val': 'Mos Eisley', 'key': 'dst_city'}], 'frame': 1, 'fromrange': False}, {'annotations': [{'val': 'Gotham City', 'key': 'or_city'}], 'frame': 1, 'fromrange': False}], 'key': 'ref'}], 'name': 'no_result'}
{'args': [], 'name': 'sorry'}
{'args': [{'val': [{'annotations': [{'val': 'Gotham City', 'key': 'or_city'}],

In [19]:
# turns, text
dial_id = 10
n_turns = len(data[dial_id]['turns']) // 2

for turn_id in range(n_turns):
    print(show_turn(dial_id, turn_id))

User   (raw):
I'd like to get away from Monday, August 15, 2016 to Wednesday, August 31, 2016. My budget is $3200 and I would leave from Detroit.

System (raw):
And what destination would you like to travel to?

User   (raw):
Anywhere, I'm looking for an adventure. What do you have?

System (raw):
How about Mexico or Porto Alegre?

User   (raw):
Let's look at both

System (raw):
Mexico is unavailable.  Porto Alegre is available, but you would have to leave from Santos.

User   (raw):
Can you please suggest some destinations that are available? I am leaving from Detroit.

System (raw):
I have no departures available from Detroit. for these dates.  Would you like to make a date change?

User   (raw):
Anything between August 15 and August 31?

System (raw):
I'm sorry, but we do not have any departures from Detroit available for these dates for the budget you provided.  Would you like to adjust your budget for this trip?



In [78]:
# turns, raw
dial = data[1].copy()
for turn_id, turn in enumerate(dial['turns']):
    turn.pop('db', None)
    print('turn_id = {}'.format(turn_id))
    pprint(turn)

turn_id = 0
{'author': 'user',
 'labels': {'active_frame': 1,
            'acts': [{'args': [{'key': 'intent', 'val': 'book'}],
                      'name': 'inform'},
                     {'args': [{'key': 'dst_city', 'val': 'Mos Eisley'},
                               {'key': 'or_city', 'val': 'Gotham City'},
                               {'key': 'budget', 'val': '2100'}],
                      'name': 'inform'},
                     {'args': [], 'name': 'greeting'}],
            'acts_without_refs': [{'args': [{'key': 'intent', 'val': 'book'}],
                                   'name': 'inform'},
                                  {'args': [{'key': 'dst_city',
                                             'val': 'Mos Eisley'},
                                            {'key': 'or_city',
                                             'val': 'Gotham City'},
                                            {'key': 'budget', 'val': '2100'}],
                                   'name': 'info

                                 'dst_city': [{'negated': False,
                                               'val': 'Kobe'}],
                                 'intent': [{'negated': False, 'val': 'book'}],
                                 'or_city': [{'negated': False,
                                              'val': 'Gotham City'}]},
                        'requests': []}]},
 'text': 'Do you have any trips from Gotham City to Kobe for my original '
         'budget of $2100?',
 'timestamp': 1471272782508.0}
turn_id = 9
{'author': 'wizard',
 'labels': {'active_frame': 5,
            'acts': [{'args': [{'key': 'duration', 'val': '3 day'},
                               {'key': 'dst_city', 'val': 'Kobe'},
                               {'key': 'or_city', 'val': 'Sapporo, Japan'},
                               {'key': 'id', 'val': 6}],
                      'name': 'suggest'}],
            'frames': [{'binary_questions': [],
                        'compare_requests': [],
       

                                         'frame': 8,
                                         'fromrange': False}]}],
                      'name': 'switch_frame'},
                     {'args': [{'key': 'duration'}], 'name': 'request'}],
            'acts_without_refs': [{'args': [{'key': 'dst_city', 'val': 'Kobe'}],
                                   'name': 'switch_frame'},
                                  {'args': [{'key': 'duration', 'val': None}],
                                   'name': 'request'}],
            'frames': [{'binary_questions': [],
                        'compare_requests': [],
                        'frame_id': 1,
                        'frame_parent_id': None,
                        'info': {'NO_RESULT': [{'negated': False, 'val': True}],
                                 'budget': [{'negated': False,
                                             'val': '2100.0'}],
                                 'dst_city': [{'negated': False,
                            

                                             'val': '2100.0'}],
                                 'dst_city': [{'negated': False,
                                               'val': 'Mos Eisley'}],
                                 'intent': [{'negated': False, 'val': 'book'}],
                                 'or_city': [{'negated': False,
                                              'val': 'Birmingham'}]},
                        'requests': []},
                       {'binary_questions': [],
                        'compare_requests': [],
                        'frame_id': 8,
                        'frame_parent_id': 7,
                        'info': {'budget': [{'negated': False,
                                             'val': '2100.0'}],
                                 'dst_city': [{'negated': False, 'val': 'Kobe'},
                                              {'negated': False,
                                               'val': 'Kobe'}],
                            

                        'compare_requests': [],
                        'frame_id': 5,
                        'frame_parent_id': 1,
                        'info': {'budget': [{'negated': False,
                                             'val': '2100.0'}],
                                 'dst_city': [{'negated': False,
                                               'val': 'Kobe'}],
                                 'intent': [{'negated': False, 'val': 'book'}],
                                 'or_city': [{'negated': False,
                                              'val': 'Gotham City'}]},
                        'requests': []},
                       {'binary_questions': [],
                        'compare_requests': [],
                        'frame_id': 6,
                        'frame_parent_id': 5,
                        'info': {'budget': [{'negated': False,
                                             'val': '2100.0'}],
                                 'dst_city': [{

                                'val': [{'annotations': [{'key': 'intent',
                                                          'val': 'book'}],
                                         'frame': 8,
                                         'fromrange': False}]}],
                      'name': 'affirm'}],
            'acts_without_refs': [{'args': [{'key': 'intent', 'val': 'book'}],
                                   'name': 'affirm'}],
            'frames': [{'binary_questions': [],
                        'compare_requests': [],
                        'frame_id': 1,
                        'frame_parent_id': None,
                        'info': {'NO_RESULT': [{'negated': False, 'val': True}],
                                 'budget': [{'negated': False,
                                             'val': '2100.0'}],
                                 'dst_city': [{'negated': False,
                                               'val': 'Mos Eisley'}],
                             

In [16]:
import string
exclude = set(string.punctuation)

def remove_punctiation(s):
    return ''.join(ch for ch in s if ch not in exclude)

# all tuples of sentence-tags pairs
sentences_tags = []

# iterate over dialogues in the data
for dialog in data:
    # iterate over turns in the dialogue
    for turn in dialog["turns"]:
        
        if turn["author"] == "wizard":
            continue
        
        # take the text from the turn and remove punctiation
        turn_sentence = remove_punctiation(turn['text'])
        
        # all slot-values in the current turn
        all_slot_values = []
        
        # iterating over acts, a list of dicts
        for act in turn['labels']['acts']:
            # list of slot-values, list of dicts
            for arg in act['args']:
                all_slot_values.append(arg)
        
        all_tags = {}
        for item in all_slot_values:
            
            # if the key is a reference, then the value
            if item["key"] != "ref":
                if "val" in item.keys():
                    if not isinstance(item["val"], list):
                        # if the slot value contains more than one word
                        if len(str(item["val"]).split()) > 1:
                            item_tokens = remove_punctiation(str(item["val"])).split()
                
                            # make all with I-
                            for item_token in item_tokens:
                                all_tags[item_token] = "I-" + item["key"]
                    
                            # change the first one with B-
                            all_tags[item_tokens[0]] = "B-" + item["key"]
                
                        else:
                            all_tags[item["val"]] = "B-" + item["key"]
            else:
                # then the "val" is a list of dictionaries
                
                for val_dict in item["val"]:
                    # annotations is a list of dictionaries
                    for annotation in val_dict["annotations"]:
                        # if the value is a key
                        if "val" in annotation.keys():
                            if len(str(annotation["val"]).split()) > 1:
                                item_tokens = remove_punctiation(str(annotation["val"])).split()
                            
                                # make all with I-
                                for item_token in item_tokens:
                                    all_tags[item_token] = "I-" + annotation["key"]
                    
                                # change the first one with B-
                                all_tags[item_tokens[0]] = "B-" + annotation["key"]
                            else:
                                all_tags[annotation["val"]] = "B-" + annotation["key"]
        
        
        sentence_tokens = turn_sentence.split()
        sentence_tags = []
        
        for token in sentence_tokens:
            if token not in all_tags.keys():
                sentence_tags.append("O")
            else:
                sentence_tags.append(all_tags[token])
        
        
        sentences_tags.append((sentence_tokens, sentence_tags))
        
pprint(len(sentences_tags))

10407


In [17]:
with open("maluuba_nlu_dataset.json", "w") as f:
    json.dump(sentences_tags, f, indent=4)

In [7]:
date = "Saturday, August 13, 2016"
print(date.split())

['Saturday,', 'August', '13,', '2016']
